<a href="https://colab.research.google.com/github/miguelAfonzalida/Cosas/blob/master/Spark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 76kB/s 
     |████████████████████████████████| 204kB 19.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=208f72a051d0d95ab25f2061e0fd2c1104df5d1362bd104722f417fc30c4e26b
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

## Autenticamos con Google Drive

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Creamos el Spark Context

In [5]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

#sqlContext = SQLContext(sc)

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Ejercicios

1. Suma de los numeros de una lista utilizando RDD

In [7]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

name 'take_photo' is not defined


In [8]:
lista = [2,4,7,9,8,1,2,0,1,2]
#La suma debería ser 36

In [9]:
type(lista)

list

In [10]:
rdd_1 = sc.parallelize(lista)

In [11]:
#Cantidad de registros
rdd_1.count()

10

En este caso cada registro es un número, cada registro podría interpretarse como una tupla de un elemento también.
Para obtener la suma de los números, osea la suma de los registros voy a utilizar REDUCE (acción) que me devuelve un único registro final resultado con el mismo formato que tenian los registros inicialmente. Es decir que con reduce voy a obtener el acumulado del las sumas y el resultado final es simplemente un registro con contenido de un número, igual formato que los 10 registros anteriores.

In [12]:
total_suma = rdd_1.reduce(lambda x,y: x+y)

In [13]:
total_suma

36

2. Valor Máximo de registros que están representados por números

In [14]:
lista = [2,4,7,9,8,1,2,0,1,2]
#El máximo sería 9

In [15]:
mayor = rdd_1.reduce(lambda x,y: x if x>y else y)
mayor

9

* Otra forma que utiliza una función para reduce

In [16]:
def max(x,y):
    if x>y:
        return x
    else:
        return y

In [17]:
rdd_1.reduce(lambda x,y: max(x,y))

9

Valor mínimo

In [18]:
rdd_1.reduce(lambda x,y: x if x<y else y)

0

* Si quiero el menor, estaría mal ordenal de de menor a mayor y pedirle el primero con Takeordered ya que estoy ordenando al pedo. Pero si me piden los tres menores entonces sí demo utilizar tekeOrdered(3).

* Los cinco menores

In [19]:
rdd_1.takeOrdered(5)

[0, 1, 1, 2, 2]

In [20]:
#Por defecto ordena de menor a mayor, osea lo anterior es lo mismo que
rdd_1.takeOrdered(5,lambda x: x)

[0, 1, 1, 2, 2]

In [22]:
#Acá si ultilizo la función lambda para oredenar de mayor a menor. 
#Si tengo tuplas especifíco por cual (o cuales) subíndice de x quiero ordenar.
rdd_1.takeOrdered(5,lambda x: -x)

[9, 8, 7, 4, 2]

## Promedio de los registros (en este caso los registros son números)

* Forma 1

### Quiero tener una tupla con el total de la suma en una componente y el total de números en la otra (para tener formato tuplas como registros y para poder contar a los registros actuales (que en este caso son solo números) a cada número le asigno el valor "1" en una tupla)

In [23]:
#Esto no se bebe hacer, aca solo estoy viendo cosas para mí
rdd_1.collect()

[2, 4, 7, 9, 8, 1, 2, 0, 1, 2]

In [24]:
#Esto no se hace, es solo para ver como queda el formato de los nuevos registros como tuplas
rdd_1.map(lambda x: (x,1)).collect()

[(2, 1),
 (4, 1),
 (7, 1),
 (9, 1),
 (8, 1),
 (1, 1),
 (2, 1),
 (0, 1),
 (1, 1),
 (2, 1)]

* ALTO: solo para aclarar. ¿Cuántos registros tengo por clave? tomando como clave a cada número:

In [25]:
rdd_1.map(lambda x: (x,1)).countByKey()

defaultdict(int, {0: 1, 1: 2, 2: 3, 4: 1, 7: 1, 8: 1, 9: 1})

In [26]:
#Quiero, para los registros con clave = 2, saber su cantidad:
rdd_1.map(lambda x: (x,1)).countByKey()[2]

3

In [27]:
#las cantidades asociadas a las claves las obtengo con values()
rdd_1.map(lambda x: (x,1)).countByKey().values()

dict_values([3, 1, 1, 1, 1, 2, 1])

In [28]:
rdd_1.map(lambda x: (x,1)).countByKey()[0]

1

In [50]:
#Podria obtener la cantidad de registros asociados a la clave "2" de otra manera, que es mucho más eficiiente
rdd_1.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).collect()

[(2, 3), (4, 1), (8, 1), (0, 1), (7, 1), (9, 1), (1, 2)]

In [52]:
#Como reduceByKey me devuelve un RDD ya que es una transformación puedo seguir aplicando transformaciones
rdd_1.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).filter(lambda x:x[0]==2).collect()

[(2, 3)]

In [59]:
rdd_1.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).filter(lambda x:x[0]==2).collect()[0]

(2, 3)

In [60]:
rdd_1.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).filter(lambda x:x[0]==2).collect()[0][1]

3

#### Ahora sigo con el cálculo del promedio (Ver que en el reduce no puedo dar nuevo formato al los registros, debo respetar las tuplas, acá en el primer comonente de la tupla pongo la suma total acumulada y en la segunda componente pongo el total de numeros que sumé)

In [39]:
rdd_1.map(lambda x: (x,1)).reduce(lambda x,y: (x[0] + y[0], x[1] + y[1] ))

(36, 10)

In [40]:
result = rdd_1.map(lambda x: (x,1)).reduce(lambda x,y: (x[0] + y[0], x[1] + y[1] ))

In [41]:
result

(36, 10)

In [42]:
result[0]

36

* Ojo reduce me devuelve solo un registro con el formato del tipo de dato de cada registro del RDD.
* Ya no tengo más al RDD tengo el tipo de dato que tenían los registros que estaba en el RDD. Por lo que a ese tipo de datos luego del "reduce" no puedo aplicar transformaciónes ni acciones

In [43]:
type(result)

tuple

In [44]:
promedio = result[0]/result[1]
promedio

3.6

### Esto está mal ojo!!!!

In [45]:
#Esto está mal!!! luego de una acción no puedo aplicar una trasformación. Además luego del Reduce ya no tengo un RDD
#rdd_1.map(lambda x: (x,1)).reduce(lambda x,y: (x[0] + y[0], x[1] + y[1] )).map(lambda x:x)

* Forma 2 (no utilizo tuplas sino que solamente obtengo la suma de los números(registros) y utilizo la acción count para luego calcular el promedio)

In [46]:
cant_reg = rdd_1.count()

In [47]:
total_suma = rdd_1.reduce(lambda x,y: x+y)

In [48]:
promedio = total_suma/cant_reg
promedio

3.6

##Map y FlatMap

In [64]:
simpleRDD = sc.parallelize([2, 3, 4])
print(simpleRDD.collect())
print("")
#Con map sigo manteniendo la misma cantidad de registros = 3, antes cada registro era un número ahora
#cada registro es una lista que tiene un "1" y ese número.
print("map")
print(simpleRDD.map(lambda x: [1, x]).collect())
print(simpleRDD.map(lambda x: (1, x)).collect())
print(simpleRDD.map(lambda x: [(1, x)]).collect())
print("")
print("flatMap ")
#Con flatMap
print(simpleRDD.flatMap(lambda x: [1, x]).collect())
print(simpleRDD.flatMap(lambda x: (1, x)).collect())
print(simpleRDD.flatMap(lambda x: [(1, x)]).collect())

[2, 3, 4]

map
[[1, 2], [1, 3], [1, 4]]
[(1, 2), (1, 3), (1, 4)]
[[(1, 2)], [(1, 3)], [(1, 4)]]

flatMap 
[1, 2, 1, 3, 1, 4]
[1, 2, 1, 3, 1, 4]
[(1, 2), (1, 3), (1, 4)]
